In [1]:
import pandas as pd

In [2]:
da_flights = pd.read_csv('da_flights.csv')

In [3]:
#Show table
da_flights

,id,origin,destination,cost
0,1,SFO,JFK,500
1,2,SFO,DFW,200
2,3,SFO,MCO,400
3,4,DFW,MCO,100
4,5,DFW,JFK,200
5,6,JFK,LHR,1000


COMPANY X employees are trying to find the cheapest flights to upcoming conferences.


When people fly long distances, a direct city-to-city flight is often more expensive than taking two flights with a stop in a hub city. Travelers might save even more money by breaking the trip into three flights with two stops. But for the purposes of this challenge, let's assume that no one is willing to stop three times. The table contains the following columns:
•   id - the unique ID of the flight;
•   origin - the origin city of the current flight;
•   destination - the destination city of the current flight;
•    cost - the cost of current flight.


Your task is to produce a trips table listing the cheapest possible trips between all origin-destination pairs present in the dataset, considering routes with up to two stops.


This table should have the columns origin, destination, and min_price (cheapest one). Sort the output table by origin, then by destination.


The cities are all represented by three-letter uppercase abbreviations.


Note: A flight from SFO to JFK is considered different from a flight from JFK to SFO.

In [4]:
# Merge flight on destination to itself from origin to create a table that show possible flight connections
flight_connection = pd.merge(da_flights, da_flights, left_on = 'destination', right_on = 'origin')

#show flight_connection
flight_connection

,id_x,origin_x,destination_x,cost_x,id_y,origin_y,destination_y,cost_y
0,1,SFO,JFK,500,6,JFK,LHR,1000
1,5,DFW,JFK,200,6,JFK,LHR,1000
2,2,SFO,DFW,200,4,DFW,MCO,100
3,2,SFO,DFW,200,5,DFW,JFK,200


In [5]:
# Add up total cost of connection flight from origin to final destination
flight_connection['total_cost'] = flight_connection['cost_x'] + flight_connection['cost_y']

#show flight_connection
flight_connection

,id_x,origin_x,destination_x,cost_x,id_y,origin_y,destination_y,cost_y,total_cost
0,1,SFO,JFK,500,6,JFK,LHR,1000,1500
1,5,DFW,JFK,200,6,JFK,LHR,1000,1200
2,2,SFO,DFW,200,4,DFW,MCO,100,300
3,2,SFO,DFW,200,5,DFW,JFK,200,400


In [6]:
# drop all unneccesary columns
flight_connection = flight_connection.drop(['id_x', 'id_y', 'origin_y', 'cost_x', 'cost_y'], axis= 1)

#Show updated dataframe
flight_connection

,origin_x,destination_x,destination_y,total_cost
0,SFO,JFK,LHR,1500
1,DFW,JFK,LHR,1200
2,SFO,DFW,MCO,300
3,SFO,DFW,JFK,400


In [7]:
# rename columns
flight_connection = flight_connection.rename(columns = {'origin_x': 'origin', 'destination_x': 'stop-over','destination_y' : 'destination', 'total_cost': 'cost' })

#show columns
flight_connection

,origin,stop-over,destination,cost
0,SFO,JFK,LHR,1500
1,DFW,JFK,LHR,1200
2,SFO,DFW,MCO,300
3,SFO,DFW,JFK,400


In [8]:
# Merge flight connection to direct flight to check possible options 
flights = pd.merge(flight_connection, da_flights, left_on = ('origin', 'destination'), right_on =('origin', 'destination'))

#show flights
flights

,origin,stop-over,destination,cost_x,id,cost_y
0,SFO,DFW,MCO,300,3,400
1,SFO,DFW,JFK,400,1,500


In [9]:
# return connection flights where it is cheaper
cheaper_flights = flights[['origin','stop-over', 'destination', 'cost_x']][flights['cost_x'] < flights['cost_y']]

# show
cheaper_flights

,origin,stop-over,destination,cost_x
0,SFO,DFW,MCO,300
1,SFO,DFW,JFK,400


In [10]:
# rename cost_x to cost
cheaper_flights = cheaper_flights.rename(columns = {'cost_x': 'cost'})

# show
cheaper_flights

,origin,stop-over,destination,cost
0,SFO,DFW,MCO,300
1,SFO,DFW,JFK,400


We can also create a table that shows only the cheapest flights either by direct or by connecting flights

In [11]:
# Join tables together by matching columns
flights = pd.concat([cheaper_flights, da_flights], ignore_index = True)

#show
flights

,origin,stop-over,destination,cost,id
0,SFO,DFW,MCO,300,NaN
1,SFO,DFW,JFK,400,NaN
2,SFO,NaN,JFK,500,1.0
3,SFO,NaN,DFW,200,2.0
4,SFO,NaN,MCO,400,3.0
5,DFW,NaN,MCO,100,4.0
6,DFW,NaN,JFK,200,5.0
7,JFK,NaN,LHR,1000,6.0


In [12]:
#fill all NaN values in stop-over column by direct flight
flights[['stop-over']] = flights[['stop-over']].fillna('direct-flight')

#show
flights

,origin,stop-over,destination,cost,id
0,SFO,DFW,MCO,300,NaN
1,SFO,DFW,JFK,400,NaN
2,SFO,direct-flight,JFK,500,1.0
3,SFO,direct-flight,DFW,200,2.0
4,SFO,direct-flight,MCO,400,3.0
5,DFW,direct-flight,MCO,100,4.0
6,DFW,direct-flight,JFK,200,5.0
7,JFK,direct-flight,LHR,1000,6.0


In [13]:
# return a table that shows cheapest flight and gives details if it is a direct flight or show stop-over location
flights[['origin', 'stop-over','destination', 'cost']].groupby(['origin', 'destination'], as_index = False).min()

,origin,destination,stop-over,cost
0,DFW,JFK,direct-flight,200
1,DFW,MCO,direct-flight,100
2,JFK,LHR,direct-flight,1000
3,SFO,DFW,direct-flight,200
4,SFO,JFK,DFW,400
5,SFO,MCO,DFW,300


In [14]:
flights[['origin', 'stop-over','destination', 'cost']].groupby(['origin', 'destination'], as_index = False).min()

,origin,destination,stop-over,cost
0,DFW,JFK,direct-flight,200
1,DFW,MCO,direct-flight,100
2,JFK,LHR,direct-flight,1000
3,SFO,DFW,direct-flight,200
4,SFO,JFK,DFW,400
5,SFO,MCO,DFW,300
